# Optimisation de l'espace des phases pour extraction lente



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xtrack as xt
from helpers import characterize_phase_space

%config InlineBackend.figure_format = "retina"

## Chargement de la maille PIMMS

In [ ]:
env = xt.load("inputs/pimms.json")
ring = env.pimms

# Alimentation des sextupoles d'extraction
env["kse1"] = 1
env["kse2"] = -6.5

## Charactérisation de l'espace des phases

Les fonctionalités du notebook précédent ont été implémentées en une fonction, pour aisance par la suite.

In [ ]:
characterize_phase_space(ring)

In [ ]:
# La chractérisation (sans visualisation) est rapide !
%time res = characterize_phase_space(ring, plot=False)

## Optimisation de la résonance

Utilisons les résultats de la function de charactérisation pour notre optimisation. Entre autres nous aimerions contrôler l'orientation de la séparatrice et la taille de la zone stable.

In [ ]:
# Dans Xsuite, il est possible de définir une Action à fournir pour les optimiseurs
class ActionSeparatrix(xt.Action):

    def __init__(self, line):
        self.line = line

    def run(self):
        return characterize_phase_space(self.line, plot=False)

In [ ]:
# Construisons et testons notre action
action = ActionSeparatrix(ring)
action.run()

In [ ]:
# Il est possible de créer des cibles d'optimization depuis l'action
# Nous agissons sur l'alimentation des sextupoles d'extraction
opt = ring.match(
    solve=False,
    method="4d",
    vary=xt.VaryList(["kse1", "kse2"], step=0.5, limits=[-7, 7]),
    targets=[
        action.target("stable_area", 1.0e-4, tol=1e-5, weight=100),
        action.target("dpx_dx_at_septum", 0.03, tol=5e-4),
    ],
)
opt.target_status()

## Utilisation d'un optimiseur externe

On pourrait obtenir la solution simplement en appelant `opt.solve()`, qui utilise l’optimiseur interne d'Xsuite.
Cependant, pour certains problèmes l'on pourrait vouloir utiliser un optimiseur différent (par exemple sans dérivées) s’il est mieux adapté au problème à résoudre.
Ici, nous montrons comment appliquer l’optimiseur Py-BOBYQA à notre problème de matching non linéaire.

In [ ]:
# Extraction d'une fonction de mérite pour l'optimiseur
merit_function = opt.get_merit_function(
    return_scalar=True,  # Py-BOBYQA veut une fonction retournant un scalaire
    check_limits=False,  # Py-BOBYQA aime explorer en-dehors des limites imposées
)

In [ ]:
# Extraction des limites et du point de départ de la fonction de mérite
bounds = merit_function.get_x_limits()
x0 = merit_function.get_x()

In [ ]:
# Recherche d'optimum en utilisant Py-BOBYQA
import pybobyqa

soln = pybobyqa.solve(
    merit_function,
    x0=x0,
    bounds=bounds.T,  # pybobyqa veut une transposée...
    rhobeg=5,
    rhoend=1e-4,
    maxfun=30,
    objfun_has_noise=True,  # <-- utile pour notre cas
    seek_global_minimum=True,
)
soln.x  # correspond à kse1, kse2

In [ ]:
# Implémentation de ces valeurs
merit_function.set_x(soln.x)

In [ ]:
# Status après optimisation
opt.target_status()
opt.vary_status()

In [ ]:
# Nous pouvons tagger la solution dans l'optimisateur d'Xsuite,
# ce qui correspond à un checkpoint ou l'on peut toujours revenir
opt.tag("bobyqa solution")

In [ ]:
# Visualisons l'espace des phases après optimisation
_ = characterize_phase_space(ring)

## Chargement d'un checkpoint de l'optimiseur

In [ ]:
opt.log()

In [ ]:
# Il est facile de charger un point spécifique
# Dans notre cas, le point de départ
opt.reload(0)
opt.vary_status()

In [ ]:
# Espace des phases correspondant au point chargé
_ = characterize_phase_space(ring)

In [ ]:
# Retour à la solution de pybobyqa
opt.reload(tag="bobyqa solution")

In [ ]:
# Espace des phases correspondant au point chargé
_ = characterize_phase_space(ring)

## Export des alimentation optimisées

In [ ]:
strengths: dict[str, float] = opt.get_knob_values()
xt.json.dump(strengths, "inputs/extraction_sextupoles.json")

---